# MiniBatch & DataLoader
- Why have to use minibatch?
  - therefore, computation at a time takes a long time and requires a lot of computation.

- 따라서 전체를 작은 배치 단위로 나누어 학습하는 개념이 등장하고 이것이 Minibatch입니다.
  - minibatch를 활용하면 1 epoch은 첫 번째 batch부터 마지막 batch까지 1회 학습한 것을 말합니다.
    - 1epoch은 전체 데이터가 한 번 전부 사용되는 것을 말하기 때문입니다.
  - minibatch의 숫자는 결국 minibatch의 크기가 결정하는데, 이 크기를 batchsize라고 합니다.
    - Batch sizes usually use a square of 2, because the memory of the CPU and GPU is 2 squares, so data transmission and reception are more efficient when it is 2 squares.
- iteration
  - 1 epoch에 필요한 minibatch 학습 횟수
    - 즉, iteration == minibatch의 개수 입니다.


## Minibatch를 위한 torch의 도구
### Dataset,DataLoader
- 이것을 통해 mini batch 학습, data shuffle, paralle까지 쉽게 수행 가능합니다.
- Dataset을 정의하고 정의한 것을 DataLoader에 전달


In [16]:
## Tensor를 입력받아 Dataset 형태로 변환해주는 TensorDataset
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset,DataLoader
torch.manual_seed(42)

In [18]:
## TensorDataset -> 기본적으로 input = tensor
x_train=torch.FloatTensor([[73, 80, 75],
                        [93, 88, 93],
                        [89, 91, 90],
                        [96, 98, 100],
                        [73, 66, 70]])
y_train=torch.FloatTensor([[152], [185], [180],[196],[142]])
dataset = TensorDataset(x_train,y_train)

In [19]:
## Dataset을 만들면, DataLoader를 사용할 수 있습니다.
### DataLoader는 기본적으로 Dataset, batch_size 2개의 파라미터를 받습니다. (shuffle 도 자주 사용합니다.)
BATCH_SIZE = 2
loader = DataLoader(dataset,batch_size = BATCH_SIZE,shuffle=True)

In [28]:
## model과 optim을 제작합니다.
model = nn.Linear(3,1)
opt = optim.SGD(model.parameters(),lr=1e-5)

In [29]:
## train을 진행합니다.
epochs = 2000
for epoch in range(1,epochs+1):
  for batch_idx, samples in enumerate(loader):
    # print(batch_idx) # 진행 과정이 궁금하면 주석 해제 후 관찰
    # print(samples) # 진행 과정이 궁금하면 주석 해제 후 관찰
    x_train,y_train = samples
    output = model(x_train)
    loss = F.mse_loss(y_train,output)
    opt.zero_grad()
    loss.backward()
    opt.step()

  if epoch % 100 == 0 :
    print(f"epoch {epoch}/{epochs} : {loss:4f}")

epoch 100/2000 : 7.521517
epoch 200/2000 : 7.082115
epoch 300/2000 : 0.000069
epoch 400/2000 : 5.743284
epoch 500/2000 : 0.074526
epoch 600/2000 : 0.281449
epoch 700/2000 : 6.664364
epoch 800/2000 : 0.143212
epoch 900/2000 : 0.008201
epoch 1000/2000 : 3.569618
epoch 1100/2000 : 0.945022
epoch 1200/2000 : 0.161745
epoch 1300/2000 : 4.214658
epoch 1400/2000 : 0.256697
epoch 1500/2000 : 3.737546
epoch 1600/2000 : 4.778416
epoch 1700/2000 : 2.054229
epoch 1800/2000 : 1.269198
epoch 1900/2000 : 1.776869
epoch 2000/2000 : 1.137710


In [30]:
## Custom Dataset
"""
Custom Dataset의 뼈대

class MyData(Dataset):
  def __init__(self): # 데이터 전처리 부분
  def __len__(self): # 데이터셋의 길이를 가져오는 부분 (len(Dataset)에서 실행되는 부분입니다.)
  def __getitem__(self,idx): #데이터셋에서 특정 1개의 샘플을 가져오는 함수 (Dataset[i]에서 실행되는 부분입니다.)
"""

'\nCustom Dataset의 뼈대\n\nclass MyData(Dataset):\n  def __init__(self): # 데이터 전처리 부분\n  def __len__(self): # 데이터셋의 길이를 가져오는 부분 (len(Dataset)에서 실행되는 부분입니다.)\n  def __getitem__(self,idx): #데이터셋에서 특정 1개의 샘플을 가져오는 함수 (Dataset[i]에서 실행되는 부분입니다.)\n'

In [32]:
## Custom Dataset을 이용한 선형회귀
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
class MyDataset(Dataset):
  def __init__(self):
      self.x_data = [[73, 80, 75],
              [93, 88, 93],
              [89, 91, 90],
              [96, 98, 100],
              [73, 66, 70]]
      self.y_data = [[152], [185], [180], [196], [142]]
  def __len__(self):
    return len(self.x_data)
  def __getitem__(self,idx):
    x = torch.FloatTensor(self.x_data[idx])
    y = torch.FloatTensor(self.y_data[idx])
    return x,y

In [33]:
dataset = MyDataset()
dataloader = DataLoader(dataset,batch_size=2,shuffle=True)
model = torch.nn.Linear(3,1)
optimizer = torch.optim.SGD(model.parameters(),lr=1e-5)

In [36]:
epochs = 2000
for epoch in range(epochs):
  for batch_idx,sample in enumerate(dataloader):
    x_train,y_train = sample
    output = model(x_train)
    loss = F.mse_loss(y_train,output)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0 :
      print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
        epoch, epochs, batch_idx+1, len(dataloader),
        loss.item()
        ))

Epoch    0/2000 Batch 1/3 Cost: 0.620303
Epoch    0/2000 Batch 2/3 Cost: 0.923488
Epoch    0/2000 Batch 3/3 Cost: 0.102913
Epoch  100/2000 Batch 1/3 Cost: 0.514346
Epoch  100/2000 Batch 2/3 Cost: 0.750052
Epoch  100/2000 Batch 3/3 Cost: 0.066901
Epoch  200/2000 Batch 1/3 Cost: 0.741161
Epoch  200/2000 Batch 2/3 Cost: 0.304402
Epoch  200/2000 Batch 3/3 Cost: 0.934136
Epoch  300/2000 Batch 1/3 Cost: 0.191779
Epoch  300/2000 Batch 2/3 Cost: 0.916370
Epoch  300/2000 Batch 3/3 Cost: 0.437480
Epoch  400/2000 Batch 1/3 Cost: 0.759907
Epoch  400/2000 Batch 2/3 Cost: 0.785733
Epoch  400/2000 Batch 3/3 Cost: 0.327191
Epoch  500/2000 Batch 1/3 Cost: 1.058382
Epoch  500/2000 Batch 2/3 Cost: 0.487384
Epoch  500/2000 Batch 3/3 Cost: 0.805354
Epoch  600/2000 Batch 1/3 Cost: 0.264247
Epoch  600/2000 Batch 2/3 Cost: 0.901120
Epoch  600/2000 Batch 3/3 Cost: 0.033232
Epoch  700/2000 Batch 1/3 Cost: 0.313617
Epoch  700/2000 Batch 2/3 Cost: 0.850789
Epoch  700/2000 Batch 3/3 Cost: 0.084397
Epoch  800/2000 